In [3]:
import pyomo.environ as pyo
from pyomo.core import Var

Cap = 250.0
ELWT = 8.0
DTE = 5.0
CEWT = 20.0
DTC = 5.0

def optimizeDualSeries():
    global data
    
    Cap_ref = 481.0
    Pow_ref = 67.00
    ratioCapMin = 0.2
    PLRMax = 1.011
    PLRMin = 0.216

    data = {None: {
        'i': {None: [0,1,2,3,4,5]},
        'j': {None: [0,1,2,3,4,5,6]},
        'a': {0: 0.762910602910603,	1: 0.0382655182655183,	2: -0.000297000297000298,	3: -0.0065280665280665,	4: 0.00000831600831600794, 5: -0.000241164241164241},
        'b': {0: 0.870954878538438,	1: -0.0386431891526463,	2: 0.00301623515115263,	3: 0.0134303412304538,	4: 0.000797720895833351,	5: -0.00161298767908122},
        'c': {0: 0.0826589317796771,	1: 0.00482250398164547,	2: -2.71346169628949E-06,	3: -0.142223753113296,	4: 2.28151869430373,	5: -1.30440787177594,	6:-0.00231757050165954},
        'capRef': {None: Cap_ref},
        'powRef': {None: Pow_ref},
        'ratioCapMin': {None: ratioCapMin},
        'plrMax': {None: PLRMax},
        'plrMin': {None: PLRMin},
        'elwt': {None: ELWT},
        'clwt': {None: CEWT+DTC},
        'cap': {None: Cap}}}

    opt = pyo.SolverFactory('ipopt')
    
    model = pyo.AbstractModel()

    model.i = pyo.Set()
    model.j = pyo.Set()
    model.a = pyo.Param(model.i)
    model.b = pyo.Param(model.i)
    model.c = pyo.Param(model.j)
    model.capRef = pyo.Param()
    model.powRef = pyo.Param()
    model.ratioCapMin = pyo.Param()
    model.plrMax = pyo.Param()
    model.plrMin = pyo.Param()
    model.elwt = pyo.Param()
    model.clwt = pyo.Param()
    model.cap = pyo.Param()
    model.ratioCap2 = pyo.Var(bounds=(ratioCapMin, None), initialize=0.5)

    def objective_function_rule(model):
        elwt1 = ELWT+model.ratioCap2*DTE
        elwt2 = ELWT
        capft1 = model.a[0]+model.a[1]*elwt1+model.a[2]*elwt1**2.0+model.a[3]*model.clwt+model.a[4]*model.clwt**2.0+model.a[5]*elwt1*model.clwt
        eirft1 = model.b[0]+model.b[1]*elwt1+model.b[2]*elwt1**2.0+model.b[3]*model.clwt+model.b[4]*model.clwt**2.0+model.b[5]*elwt1*model.clwt
        plr1 = model.cap*(1-model.ratioCap2)/(model.capRef*capft1)
        eirfplr1 = model.c[0]+model.c[1]*model.clwt+model.c[2]*model.clwt**2.0+model.c[3]*plr1+model.c[4]*plr1**2.0+model.c[5]*plr1**3.0+model.c[6]*model.clwt*plr1
        Pow1 = model.powRef*capft1*eirft1*eirfplr1
        capft2 = model.a[0]+model.a[1]*elwt2+model.a[2]*elwt2**2.0+model.a[3]*model.clwt+model.a[4]*model.clwt**2.0+model.a[5]*elwt2*model.clwt
        eirft2 = model.b[0]+model.b[1]*elwt2+model.b[2]*elwt2**2.0+model.b[3]*model.clwt+model.b[4]*model.clwt**2.0+model.b[5]*elwt2*model.clwt
        plr2 = model.cap*model.ratioCap2/(model.capRef*capft2)
        eirfplr2 = model.c[0]+model.c[1]*model.clwt+model.c[2]*model.clwt**2.0+model.c[3]*plr2+model.c[4]*plr2**2.0+model.c[5]*plr2**3.0+model.c[6]*model.clwt*plr2
        Pow2 = model.powRef*capft2*eirft2*eirfplr2
        Pow = Pow1+Pow2
        return Pow

    def constraint_1_rule(model):
        return model.ratioCap2 >= model.ratioCapMin

    def constraint_2_rule(model):
        return model.ratioCap2 <= 1.0-model.ratioCapMin    

    def constraint_3_rule(model):
        elwt1 = ELWT+model.ratioCap2*DTE
        capft1 = model.a[0]+model.a[1]*elwt1+model.a[2]*elwt1**2.0+model.a[3]*model.clwt+model.a[4]*model.clwt**2.0+model.a[5]*elwt1*model.clwt
        return model.plrMax*(model.capRef*capft1) >= model.cap*(1-model.ratioCap2)

    def constraint_4_rule(model):
        elwt2 = ELWT
        capft2 = model.a[0]+model.a[1]*elwt2+model.a[2]*elwt2**2.0+model.a[3]*model.clwt+model.a[4]*model.clwt**2.0+model.a[5]*elwt2*model.clwt
        return model.plrMax*(model.capRef*capft2) >= model.cap*model.ratioCap2 

    def constraint_5_rule(model):
        elwt1 = ELWT+model.ratioCap2*DTE
        capft1 = model.a[0]+model.a[1]*elwt1+model.a[2]*elwt1**2.0+model.a[3]*model.clwt+model.a[4]*model.clwt**2.0+model.a[5]*elwt1*model.clwt
        return model.plrMin*(model.capRef*capft1) <= model.cap*(1-model.ratioCap2)

    def constraint_6_rule(model):
        elwt2 = ELWT
        capft2 = model.a[0]+model.a[1]*elwt2+model.a[2]*elwt2**2.0+model.a[3]*model.clwt+model.a[4]*model.clwt**2.0+model.a[5]*elwt2*model.clwt
        return model.plrMin*(model.capRef*capft2) <= model.cap*model.ratioCap2 

    model.objective_function = pyo.Objective(rule = objective_function_rule, sense = pyo.minimize)
    model.constraint_1 = pyo.Constraint(rule = constraint_1_rule)
    model.constraint_2 = pyo.Constraint(rule = constraint_2_rule)
    model.constraint_3 = pyo.Constraint(rule = constraint_3_rule)
    model.constraint_4 = pyo.Constraint(rule = constraint_4_rule)
    model.constraint_5 = pyo.Constraint(rule = constraint_5_rule)
    model.constraint_6 = pyo.Constraint(rule = constraint_6_rule)

    instance = model.create_instance(data)
    results = opt.solve(instance)

    print('Solver status: '+results.solver.status+' Termination status: '+results.solver.termination_condition)
    for v in instance.component_objects(Var, active=True):
        print ("Variable",v)
        varobject = getattr(instance, str(v))
        for index in varobject:
            if index==None:
                print ("   ", "   ", varobject[index].value)
                ratioCap_opt = varobject[index].value
            else:
                print ("   ",index, varobject[index].value)
    return ratioCap_opt
            
def Chiller(elwt, cap):
    a=data[None]['a']
    b=data[None]['b']
    clwt = data[None]['clwt'][None]
    capRef = data[None]['capRef'][None]
    c=data[None]['c']
    powRef=data[None]['powRef'][None]
    plrMax=data[None]['plrMax'][None]
    plrMin=data[None]['plrMin'][None]
    capft = a[0]+a[1]*elwt+a[2]*elwt**2.0+a[3]*clwt+a[4]*clwt**2.0+a[5]*elwt*clwt
    eirft = b[0]+b[1]*elwt+b[2]*elwt**2.0+b[3]*clwt+b[4]*clwt**2.0+b[5]*elwt*clwt
    plr = cap/(capRef*capft)
    eirfplr = c[0]+c[1]*clwt+c[2]*clwt**2.0+c[3]*plr+c[4]*plr**2.0+c[5]*plr**3.0+c[6]*clwt*plr
    if (plr >= plrMax*1.02) or (plr <= plrMin*0.98):
        Pow =  powRef*100.0
    else:
        Pow =  powRef*capft*eirft*eirfplr
    return Pow, plr

def ChillerDualSeriesParalel(elwt, cap, ratioCap2):
    elwt1 = elwt+ratioCap2*DTE
    elwt2 = elwt
    cap1 = cap*(1-ratioCap2)
    cap2 = cap*ratioCap2
    Pow1, plr1 = Chiller(elwt1, cap1)
    Pow2, plr2 = Chiller(elwt2, cap2)
    Pow = Pow1 + Pow2
    return Pow, plr1, plr2

def optimize():
    ratioCap_opt = optimizeDualSeries()
    Pow_opt, plr1_opt, plr2_opt = ChillerDualSeriesParalel(ELWT, Cap, ratioCap_opt)
    Pow, plr = Chiller(ELWT, Cap)

    if Pow_opt<Pow: 
        n_series = 2
        ratioCap_series = ratioCap_opt
    else:
        n_series = 1
        ratioCap_series = 0.0
    return n_series, ratioCap_series

n_series, ratioCap2_series = optimize()

Solver status: ok Termination status: optimal
Variable ratioCap2
        0.4755024910414412


In [4]:
print(n_series, ratioCap2_series)

2 0.4755024910414412
